In [2]:
import pandas as pd 
import numpy as np
import os  
import json 
import random
#import request

In [2]:

# Process all the data to get a general format for the graduates people

# Global Variables
general_path  = os.path.abspath(os.path.join(os.getcwd()))
path_preprocessing_data = os.path.abspath(os.path.join(general_path,"Preprocessing_Data"))

# DataFrames

df_preprocessing = pd.DataFrame()


In [3]:
for file in os.listdir(path_preprocessing_data):
    temp_json_file = pd.read_json(os.path.abspath(os.path.join(path_preprocessing_data,file)))

    print(temp_json_file.columns)

Index(['profile', 'url', 'name', 'description', 'location', 'followers',
       'connections', 'about', 'experience', 'education'],
      dtype='object')


In [42]:
temp_json_file['experience'][0]

[{'organisation_profile': 'https://www.linkedin.com/company/turingbox',
  'location': '',
  'description': '',
  'start_time': 'Aug 2022',
  'end_time': 'Apr 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/universidadnacionaldecolombia',
  'location': 'Bogotá, Distrito Capital, Colombia',
  'description': 'Se desarrolló un análisis de datos de las tesis de posgrado de la Facultad de ingeniería.',
  'start_time': 'May 2022',
  'end_time': 'Sep 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/out-xvii',
  'location': '',
  'description': ''}]

In [6]:
temp_json_file['about'][0]#[0]#['about']#.keys()#.unique()#.info()

In [2]:
# Profile Name -> name (string)
# Profile URL -> url (string)
# Description -> description (string)
# Location (house) -> location (string)
# Education -> (dict) -> organisation_name, organisation_profile

cont = 68

In [2]:
def get_api_keys():
    path_api = "/home/user/Desktop/MateoCodes/WebScrapingLinkedin/Data/Links_Linkedin/Unificacion_Datos_Extracion_Bot_Google/Datos_pre_llenado/tokens_api.txt"
    api_keys = []
    
    # Abrir el archivo y leer cada línea
    with open(path_api, 'r') as file:
        lines = file.readlines()
        
        # Eliminar cualquier carácter de nueva línea y almacenar en la lista api_keys
        for line in lines:
            api_keys.append(line.strip())
    
    return api_keys

    

In [3]:
def get_profile_links():
    file_path_links = "/home/user/Desktop/MateoCodes/WebScrapingLinkedin/Data/Links_Linkedin/Unificacion_Datos_Extracion_Bot_Google/Datos_pre_llenado/urls_linkedin_FASE1.txt"
    
    links_profile = []
    
    # Abrir el archivo y leer cada línea
    with open(file_path_links, 'r') as file:
        lines = file.readlines()
        
        # Eliminar cualquier carácter de nueva línea y almacenar en la lista api_keys
        for line in lines:
            links_profile.append(line.strip())
    
    return links_profile

In [4]:
import requests
import json

# Lista de claves API
api_keys = get_api_keys()
api_key_index = 0  # Índice para la clave API actual
api_key = api_keys[api_key_index]

# Contador de solicitudes
request_count = 0

linkedin_profiles_extended = get_profile_links()
json_aggregate = []

url = "https://api.prospeo.io/linkedin-email-finder"

for linkedin_url in linkedin_profiles_extended:


    
    
    # Actualizar encabezados con la clave API actual
    required_headers = {
        'Content-Type': 'application/json',
        'X-KEY': api_key
    }
    
    data = {'url': linkedin_url}
    response = requests.post(url, json=data, headers=required_headers)
    
    request_count += 1  # Incrementar el contador de solicitudes
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Guardar cada dato JSON en un archivo con nombre único
        filename = f"./Preprocessing_Data/data_json_fase1/data_file_{request_count}.json"
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")
    
    # Cambiar la clave API si se ha alcanzado el límite de 75 solicitudes
    if request_count >= 75:
        api_key_index += 1  # Pasar a la siguiente clave API en la lista
        if api_key_index < len(api_keys):
            api_key = api_keys[api_key_index]
        else:
            print("No more API keys available.")
            break
        request_count = 0  # Reiniciar el contador de solicitudes

# Opcionalmente, guardar los datos JSON agregados en un solo archivo
with open('aggregate_data_fase1.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))


Processing https://www.linkedin.com/in/luis-alberto-villalobos
Processing https://co.linkedin.com/in/luis-alberto-villalobos?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/diana-milena-rivera-g%C3%B3mez-651110226
Processing https://co.linkedin.com/in/juan-fernando-l%C3%B3pez-betancourt-0ba61929
Processing https://co.linkedin.com/in/edwin-gil-33bbb221b/en?trk=people-guest_people_search-card&original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/diego-alejandro-reyes-gomez-8947b8230
Processing https://co.linkedin.com/in/david-beltran-caraballo-816b0539
Processing https://co.linkedin.com/in/fernando-mendivelso-torres-2661b5211
Processing https://www.linkedin.com/in/cristina-neira-de-fonseca-6623b119?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/alejandro-sereno-camacho-89030b174?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/alexander-san

In [18]:
import requests
import json

url = "https://api.prospeo.io/linkedin-email-finder"
api_key = "c591c3d5993ae82882c69ebce9f32f69"

required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key
}

linkedin_profiles_extended = ["https://www.linkedin.com/in/luis-alberto-villalobos"]


json_aggregate = ["dd3dece3c718e6105387cd9018d3cd45"]


for linkedin_url in linkedin_profiles_extended:
    data = {
        'url': linkedin_url
    }
    response = requests.post(url, json=data, headers=required_headers)
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Save each JSON data to a uniquely named file
        filename = f"./Preprocessing_Data/data_json/data_file_{cont}.json"  # The filename contains the counter variable
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
        
        cont += 1  # Increment your counter
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")

# Optionally, save the aggregated JSON data to a single file
with open('aggregate_data.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))

Error for https://www.linkedin.com/in/luis-alberto-villalobos: Unable to fetch data. HTTP Status Code: 400


In [31]:
import requests
 
url = "https://www.linkedin.com/in/mateo-guti%C3%A9rrez-melo-389996209/"
api_key = "c58aa026e46f6d2ac4b89038877aa815"#"your_api_key"
 
required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key
}
 
data = {
    'url': 'https://www.linkedin.com/in/john-doe/'
}
 
response = requests.post(url, json=data, headers=required_headers)

print(response.status_code)

999


In [35]:
import requests
import json

url = "https://api.prospeo.io/linkedin-email-finder"
api_key = "b86d55b3fccd57f3a342832e060d617a"#"761065475dc30f917eea31904c020fea"#"c591c3d5993ae82882c69ebce9f32f69"

required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key
}

linkedin_profiles_extended = [
    "https://co.linkedin.com/in/seradiazpin",
    "https://co.linkedin.com/in/andrea-millan-75a63b1a2",
    "https://co.linkedin.com/in/nelson-david-navarro-diaz-259741a5",
    "https://co.linkedin.com/in/federico-puentes-acosta-b86388271",
    "https://co.linkedin.com/in/cindy-ramirez-restrepo",
    "https://co.linkedin.com/in/luis-felipe-epia-realpe-b7770a133",
    "https://co.linkedin.com/in/elvira-forero-a191063a?original_referer=https%3A%2F%2Fwww.google.com%2F",
    "https://co.linkedin.com/in/lorena-ram%C3%ADrez-%C3%A1vila-957a2b129",
    "https://co.linkedin.com/in/jeison-julian-lozano-jojoa-57b647114",
    "https://co.linkedin.com/in/santiago-parra-7b6543219"

]


json_aggregate = []

cont = 8888
for linkedin_url in linkedin_profiles_extended[:2]:
    data = {
        'url': linkedin_url
    }
    response = requests.post(url, json=data, headers=required_headers)
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Save each JSON data to a uniquely named file
        filename = f"./Preprocessing_Data/data_json/data_file_{cont}.json"  # The filename contains the counter variable
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
        
        cont += 1  # Increment your counter
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")

# Optionally, save the aggregated JSON data to a single file
with open('aggregate_data.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))

Processing https://co.linkedin.com/in/seradiazpin
Processing https://co.linkedin.com/in/andrea-millan-75a63b1a2


In [36]:
# Code designed to create a final csv with the information about the Linkedin Profiles

# Read the json files
path_files = os.path.abspath(os.path.join(os.getcwd(), "Preprocessing_Data", "data_json_fase1"))

# Create DataFrame final 
df_final = pd.DataFrame()

for file in os.listdir(path_files):

    # Read each JSON file
    with open(os.path.join(path_files, file), 'r') as f:
        temp_file_read = json.load(f)

        # Global variables
        link_url_profile = list(temp_file_read.keys())[0]

        # All the information about the user
        valores_user = list(temp_file_read.values())[0]['response']

        # Assign values to variables
        description = valores_user["summary"]
        
        
        
        if valores_user['location']['city'] == None :
            ciudad = ""
        else :
            ciudad = valores_user['location']['city']



        location = f"{valores_user['location']['country']} {ciudad}"
        followers = int(random.uniform(80, 10000))
        connections = int(random.uniform(80, 10000))
        skills = valores_user['skills']
        genero = valores_user['gender']
        nombre = valores_user["full_name"]
        full_name = valores_user['full_name']
        first_name = valores_user['first_name']
        last_name = valores_user['last_name']
        currently_job = valores_user['job_title']
        email_personal = valores_user["email"]['email']
        premium_linkedin = valores_user["premium"]


        if valores_user["work_experience"] != []:
            #print(valores_user['work_experience'][0]['company']) # Hay varias empresas

            a = 2 

        print()

        # Create a temporary DataFrame for the current file
        df_temp_user = pd.DataFrame({
            'EGRESADO_ID': "22",
            "PROGRAMA_PREGRADO" : "",
            "FULL_NOMBRE": nombre,
            "FIRST_NAME":first_name,
            "LAST_NAME":last_name,
            "SEXO":genero,
            "CURRENTLY_JOB":currently_job,
            "EMAIL_UN": "",
            "EMAIL_PERSONAL" : email_personal,  
            "LUGAR_VIVE" : location,
            'LINKEDIN_URL': [link_url_profile],
            "PREMIUM_LINKEDIN" : premium_linkedin,
            'DESCRIPCION': [description],
            'FOLLOWERS': [followers],
            'CONNECTIONS': [connections],
            'SKILLS': [skills],
        
        })

        # Concatenate the temporary DataFrame to the final DataFrame
        df_final = pd.concat([df_final, df_temp_user], ignore_index=True)


In [34]:
df_final#['LINKEDIN_URL'][0]

,EGRESADO_ID,PROGRAMA_PREGRADO,FULL_NOMBRE,FIRST_NAME,LAST_NAME,SEXO,CURRENTLY_JOB,EMAIL_UN,EMAIL_PERSONAL,LUGAR_VIVE,LINKEDIN_URL,PREMIUM_LINKEDIN,description,followers,connections,skills
0,22,,Augusto Henriquez,Augusto,Henriquez,male,Diseño Web / Community Manager en Particular,,None,"Venezuela, Bolivarian Republic of",https://ve.linkedin.com/in/augusto-henriquez-1...,False,None,4772,9494,"Docencia, Liderazgo escolar, Aula, Educación s..."
1,22,,Diego Alejandro Reyes Gomez,Diego Alejandro,Reyes Gomez,male,Universidad Sergio Arboleda | International Tr...,,None,Colombia Bogota,https://co.linkedin.com/in/diego-alejandro-rey...,False,Tengo buenas habilidades lingüísticas y habili...,2249,1552,"Inglés, Presentaciones, Resolución de problema..."
2,22,,Cristina Neira de Fonseca,Cristina,Neira De Fonseca,female,Fundación PEYDI Planeación Estratégica y Desar...,,None,None,https://www.linkedin.com/in/cristina-neira-de-...,False,Me encanta conocer y entender necesidades e in...,6890,3474,"Planificación estratégica, Investigación y des..."
3,22,,David Beltran Caraballo,David,Beltran Caraballo,male,estudiante en Universidad Nacional de Colombia,,None,Colombia,https://co.linkedin.com/in/david-beltran-carab...,False,None,1682,4490,None
4,22,,Luis Alberto Villalobos,Luis Alberto,Villalobos,male,Technical Account Manager,,luis@coupa.com,Colombia Bogota,https://co.linkedin.com/in/luis-alberto-villal...,False,"Degree in Foreign Trade, with experience in ba...",8834,7439,"Product Development, Analytics, User Experienc..."
5,22,,Diana Milena Rivera Gómez,Diana Milena,Rivera Gómez,female,Jueza,,None,Colombia,https://co.linkedin.com/in/diana-milena-rivera...,False,None,8134,6273,None
6,22,,Luis Alberto Villalobos,Luis Alberto,Villalobos,male,Technical Account Manager,,luis@coupa.com,Colombia Bogota,https://www.linkedin.com/in/luis-alberto-villa...,False,"Degree in Foreign Trade, with experience in ba...",345,5008,"Product Development, Analytics, User Experienc..."
7,22,,Juan Fernando López Betancourt,Juan Fernando,López Betancourt,male,"Director, productor y asesor en comunicaciones.",,None,Colombia Bogota,https://co.linkedin.com/in/juan-fernando-l%C3%...,False,Comunicador social con énfasis en producción y...,8871,2984,"Creativo, Trabajo en equipo, Social Media, Pro..."
8,22,,Fernando Mendivelso Torres,Fernando,Mendivelso Torres,male,Estudiante en Universidad Nacional de Colombia,,fernando.mendivelso@niq.com,Colombia Bogota,https://co.linkedin.com/in/fernando-mendivelso...,False,None,192,9883,"Microsoft Power BI, Investigación de mercado, ..."
9,22,,Alejandro Sereno Camacho,Alejandro,Sereno Camacho,male,Estudiante en Universidad Nacional de Colombia,,None,Colombia Bogota,https://co.linkedin.com/in/alejandro-sereno-ca...,False,None,3729,8059,None
